In [1]:
import pandas as pd

# 读取数据
financial_ratio = pd.read_csv('/Users/shawn/Github/M1/金融機構與風險管理/S_P500 財務比率.csv')
rating = pd.read_csv('/Users/shawn/Github/M1/金融機構與風險管理/rating.csv')

financial_ratio['datadate'] = pd.to_datetime(financial_ratio['datadate'], format='%Y/%m/%d')
financial_ratio['datadate'] = financial_ratio['datadate'].dt.strftime('%Y-%m-%d')


rating['datadate'] = pd.to_datetime(rating['datadate'], format='%Y-%m-%d')
rating['datadate'] = rating['datadate'].dt.strftime('%Y-%m-%d')

In [2]:
merged_df = pd.merge(financial_ratio, rating[['datadate', 'ticker', 'splticrm']], on=['datadate', 'ticker'], how='left')
merged_df['splticrm']


print(merged_df[merged_df['ticker'] == 'AMD'][['datadate','ticker','splticrm']].head()) 
print(rating[rating['ticker'] == 'AMD'][['datadate','ticker','splticrm']].head()) 

         datadate ticker splticrm
31998  2010-01-31    AMD       B-
31999  2010-02-28    AMD       B-
32000  2010-03-31    AMD       B-
32001  2010-04-30    AMD       B-
32002  2010-05-31    AMD       B-
       datadate ticker splticrm
388  2001-01-31    AMD        B
389  2001-02-28    AMD        B
390  2001-03-31    AMD        B
391  2001-04-30    AMD        B
392  2001-05-31    AMD        B


In [3]:
# financial_ratio, rating期間範圍無對齊
merged_df.dropna(subset=['splticrm'], inplace=True)
def filter_rows(group):
    # 組內rating前後不同的才會被留下
    mask = group['splticrm'] != group['splticrm'].shift()
    return group[mask]
merged_df = merged_df.groupby('ticker').apply(filter_rows).reset_index(drop=True)
merged_df['ticker'].value_counts().to_frame() 


,count
ticker,
CHK,12
AKS,7
ETFC,7
M,7
DO,7
...,...
JNJ,1
KEY,1
KMB,1


In [4]:
# merged_df保留了所有ticker
merged_df2 = merged_df.groupby('ticker').filter(lambda x: len(x) > 1)
print(merged_df2['ticker'].value_counts().to_frame())
print(len(merged_df2))
merged_df2.to_csv('SP500_merge.csv')
merged_df2.head()

        count
ticker       
CHK        12
ETFC        7
DO          7
AKS         7
M           7
...       ...
OKE         2
D           2
ODP         2
DD          2
PEG         2

[225 rows x 1 columns]
668


,gvkey,permno,adate,qdate,datadate,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,ticker,cusip,splticrm
1,1078,20482,2008/12/31,2009/9/30,2010-01-31,23.252,0.279,10.335,14.584,14.706,...,0.091,0.000,0.0,-0.031,3.838,0.370,3.02%,ABT,282410,AA
2,1078,20482,2011/12/31,2012/6/30,2012-10-31,22.742,0.242,11.602,20.092,20.342,...,0.115,0.000,0.0,-0.099,4.198,NaN,3.11%,ABT,282410,A+
3,1078,20482,2015/12/31,2016/9/30,2017-01-31,35.311,0.334,14.313,24.145,24.285,...,0.070,0.000,0.0,-0.046,3.460,2.802,2.54%,ABT,282410,BBB
4,12540,75510,2009/11/30,2009/11/30,2010-01-31,27.386,0.281,19.497,40.886,41.410,...,0.192,0.023,0.0,-0.112,3.292,5.746,NaN,ADBE,00724F10,BBB+
5,12540,75510,2012/11/30,2013/8/31,2013-12-31,42.614,0.312,22.531,64.386,65.801,...,0.196,0.023,0.0,-0.086,4.148,4.301,NaN,ADBE,00724F10,A-


In [5]:
ratings_map = {
    'D': 0, 'CC': 1, 'CCC': 2, 'CCC+': 3,
    'B-': 5, 'B': 6, 'B+': 7, 'BB-': 8, 'BB': 9, 'BB+': 10,
    'BBB-': 12, 'BBB': 14, 'BBB+': 16,
    'A-': 19, 'A': 22, 'A+': 25, 'AA-': 28, 'AA': 32, 'AA+': 36, 'AAA': 40
}

# 將評級參數化
merged_df2['rating'] = merged_df2['splticrm'].map(ratings_map)
merged_df2.head()


,gvkey,permno,adate,qdate,datadate,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,ticker,cusip,splticrm,rating
1,1078,20482,2008/12/31,2009/9/30,2010-01-31,23.252,0.279,10.335,14.584,14.706,...,0.000,0.0,-0.031,3.838,0.370,3.02%,ABT,282410,AA,32
2,1078,20482,2011/12/31,2012/6/30,2012-10-31,22.742,0.242,11.602,20.092,20.342,...,0.000,0.0,-0.099,4.198,NaN,3.11%,ABT,282410,A+,25
3,1078,20482,2015/12/31,2016/9/30,2017-01-31,35.311,0.334,14.313,24.145,24.285,...,0.000,0.0,-0.046,3.460,2.802,2.54%,ABT,282410,BBB,14
4,12540,75510,2009/11/30,2009/11/30,2010-01-31,27.386,0.281,19.497,40.886,41.410,...,0.023,0.0,-0.112,3.292,5.746,NaN,ADBE,00724F10,BBB+,16
5,12540,75510,2012/11/30,2013/8/31,2013-12-31,42.614,0.312,22.531,64.386,65.801,...,0.023,0.0,-0.086,4.148,4.301,NaN,ADBE,00724F10,A-,19


In [6]:
def process_data(group):
    # 保留前一期的評價
    group['prev_rating'] = group['rating'].shift(1)
    group['rating_diff'] = group['rating'] - group['rating'].shift(1)
    # 變化率
    for col in group.columns:
        if pd.api.types.is_numeric_dtype(group[col]):
            group[col + '_change'] = group[col].pct_change()

    # 刪除前一期評價(prev_rating)
    # group = group[group['is_upgraded'] == 1].drop(columns=['prev_rating'])
    group = group.dropna(subset=['prev_rating'])

    return group

In [7]:
merged_df3 = merged_df2.groupby('ticker').apply(process_data).reset_index(drop=True)
merged_df3[['ticker','prev_rating','splticrm','rating_diff']].head()
merged_df3.head()

,gvkey,permno,adate,qdate,datadate,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,sale_nwc_change,rd_sale_change,adv_sale_change,staff_sale_change,accrual_change,ptb_change,PEG_trailing_change,rating_change,prev_rating_change,rating_diff_change
0,1078,20482,2011/12/31,2012/6/30,2012-10-31,22.742,0.242,11.602,20.092,20.342,...,-0.145647,0.263736,NaN,NaN,2.193548,0.093799,0.000000,-0.218750,NaN,NaN
1,1078,20482,2015/12/31,2016/9/30,2017-01-31,35.311,0.334,14.313,24.145,24.285,...,0.270724,-0.391304,NaN,NaN,-0.535354,-0.175798,6.572973,-0.440000,-0.21875,0.571429
2,12540,75510,2012/11/30,2013/8/31,2013-12-31,42.614,0.312,22.531,64.386,65.801,...,-0.232854,0.020833,0.000000,NaN,-0.232143,0.260024,-0.251479,0.187500,NaN,NaN
3,1632,60871,2009/10/31,2010/7/31,2010-09-30,20.402,0.346,9.867,15.458,15.769,...,0.108559,-0.166667,0.000000,NaN,0.368421,-0.004239,NaN,0.187500,NaN,NaN
4,1632,60871,2015/10/31,2016/4/30,2016-07-31,28.032,0.289,15.494,21.935,22.240,...,-0.091337,0.032432,-0.666667,NaN,-0.500000,0.285527,0.000000,-0.263158,0.18750,-2.666667


In [8]:
merged_df3 = merged_df3.drop(columns=['gvkey','permno','adate','qdate','rating_diff_change','prev_rating_change','rating_change'])
merged_df3.to_csv('/Users/shawn/Github/M1/金融機構與風險管理/SP500_change_V3.csv')

In [9]:
merged_df3[['splticrm','rating','prev_rating','rating_diff']]

,splticrm,rating,prev_rating,rating_diff
0,A+,25,32.0,-7.0
1,BBB,14,25.0,-11.0
2,A-,19,16.0,3.0
3,A-,19,16.0,3.0
4,BBB,14,19.0,-5.0
...,...,...,...,...
438,A-,19,16.0,3.0
439,BBB+,16,19.0,-3.0
440,BBB-,12,14.0,-2.0
441,BBB,14,12.0,2.0
